In [2]:
import kagglehub
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")


In [4]:
# 安裝必要套件（只需跑一次）
!pip install xgboost scikit-learn

# 匯入套件
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import xgboost as xgb

# 載入資料（用你的 Kaggle 資料或上傳 CSV）
data = pd.read_csv(f"{path}/creditcard.csv")


# 預處理
data['Class'] = data['Class'].astype(int)
data = data.drop(['Time'], axis=1)
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

# 分開 feature & label
X = data.drop(columns=['Class'])
y = data['Class']

# 建立 IsolationForest 並訓練
iso_model = IsolationForest(n_estimators=100, contamination=0.0015, random_state=42)
iso_model.fit(X)

# 取得 anomaly score 並加入 X 特徵中
anomaly_score = iso_model.decision_function(X)  # 越小越異常
X['anomaly_score'] = anomaly_score

# 分割訓練集 / 測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 建立 XGBoost 分類器
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

# 預測與評估
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:18:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.93      0.76      0.84       148

    accuracy                           1.00     85443
   macro avg       0.96      0.88      0.92     85443
weighted avg       1.00      1.00      1.00     85443

